In [20]:
import numpy as np 
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization, LSTM

import string
from preprocessor import load_sonnets

sonnets = load_sonnets()

In [81]:
chars = list(string.ascii_lowercase)
#chars = list(string.ascii_letters)
chars += [",", ".", "?", "!", ":", ";", "'", "(", ")", " ", "\n", "-"]

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

train_strings = []
char_length = 41
char_skip = 5 #20

for _, s in enumerate(sonnets):
    #sonnet = "\n".join(sonnets[0]).lower()
    sonnet = "\n".join(s).lower()
    
    for i in range(0, len(sonnet) - char_length, char_skip):
        train_strings.append(sonnet[i: i+char_length])
    #break
    
x = np.zeros((len(train_strings), char_length - 1, len(chars)), dtype=np.bool)
y = np.zeros((len(train_strings), len(chars)), dtype=np.bool)

for i, train_string in enumerate(train_strings):
    for t, char in enumerate(train_string[:-1]):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[train_string[-1]]] = 1

In [84]:
#timesteps = 8
model = Sequential()
#model.add(Dense(200, input_shape=(200,)))
model.add(LSTM(200, input_shape=(char_length - 1, len(chars))))
#model.add(Dense(200))
#model.add(Activation('relu'))
#model.add(Dropout(dropout))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [86]:
#model.summary()
model.fit(x, y, batch_size=64, epochs=10)

Epoch 1/5
17628/17628 [==============================] - 43s 2ms/step - loss: 2.0690 - acc: 0.3883
Epoch 2/5
17628/17628 [==============================] - 43s 2ms/step - loss: 2.0016 - acc: 0.4021
Epoch 3/5
17628/17628 [==============================] - 43s 2ms/step - loss: 1.9412 - acc: 0.4203
Epoch 4/5
17628/17628 [==============================] - 43s 2ms/step - loss: 1.8789 - acc: 0.4366
Epoch 5/5
17628/17628 [==============================] - 43s 2ms/step - loss: 1.8206 - acc: 0.4532


In [50]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [87]:
seed = "shall i compare thee to a summer's day?\n"
generated = ' ' + seed

for i in range(100):
    x_pred = np.zeros((1, char_length - 1, len(chars)))
    for t, char in enumerate(seed):
        x_pred[0, t, char_indices[char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
    
    next_index = sample(preds, temperature=0.5)
    next_char = indices_char[next_index]
    generated += next_char
    seed = seed[1:] + next_char

print(generated)

 shall i compare thee to a summer's day?
out then no ben thee her that then thye may,
oun thes not thee my lover shere theer in thee,
wher my


In [82]:
print(y.shape)

(17628, 38)
